# Team Members:
## Godhani Harsh

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession\
    .builder\
    .appName("BigDataCovid19")\
    .getOrCreate()

In [3]:
filename = r"time_series_19-covid-Confirmed_archived_0325.csv"
# https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html
df = spark.read.load(filename, format="csv", sep=",", inferSchema="true", header="true")
# df.printSchema()
df.toPandas()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,None,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,82,114,147,177,212,272,322,411,599,599.0
1,None,Japan,36.0000,138.0000,2,1,2,2,4,4,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,None,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,212,226,243,266,313,345,385,432,455,455.0
3,None,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,None,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,None,Jersey,49.1900,-2.1100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
497,None,Puerto Rico,18.2000,-66.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
498,None,Republic of the Congo,-1.4400,15.5560,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
499,None,The Bahamas,24.2500,-76.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [4]:
df = df.na.drop()

In [5]:
df.describe().toPandas()

,summary,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,count,135,135,135,135,135,135,135,135,135,...,135,135,135,135,135,135,135,135,135,135
1,mean,None,None,31.527312592592594,-8.28564814814814,4.059259259259259,4.762962962962963,6.82962962962963,10.437037037037037,15.42962962962963,...,683.5259259259259,692.0518518518519,723.8518518518518,751.837037037037,783.2666666666667,847.2444444444444,917.5777777777778,995.6296296296297,1080.0296296296297,1079.4518518518519
2,stddev,None,None,21.685687608153504,100.36093612126233,38.25713531398141,38.35754949283951,47.69391536972654,66.31005950197404,92.60069474778388,...,5838.050280023636,5838.15536743467,5851.67628301258,5860.767767450878,5876.640657798937,5908.195565719372,5957.689161184351,6026.642542860333,6124.55748135317,6124.684068268117
3,min,Alabama,Australia,-41.4545,-157.4983,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,max,Zhejiang,United Kingdom,71.7069,165.618,444,444,549,761,1058,...,67790,67794,67798,67799,67800,67800,67800,67800,67800,67800


In [6]:
df_aus = df.filter(df["Country/Region"]=="Australia")
df_aus.select('Province/State').toPandas()

,Province/State
0,New South Wales
1,Victoria
2,Queensland
3,South Australia
4,From Diamond Princess
5,Western Australia
6,Tasmania
7,Northern Territory
8,Australian Capital Territory


In [7]:
confirm_cases = df_aus.select(df_aus.columns[4:]).groupBy().sum().rdd.flatMap(lambda x: x).collect()
days = [*range(len(confirm_cases))]
new_data = list(zip(days, confirm_cases))

In [8]:
df_schema = StructType([StructField("day", IntegerType())
                      ,StructField("cases", IntegerType())])
df_new = spark.createDataFrame(new_data,schema=df_schema) 
df_new.show()
#https://www.programcreek.com/python/example/104715/pyspark.sql.types.StructType

+---+-----+
|day|cases|
+---+-----+
|  0|    0|
|  1|    0|
|  2|    0|
|  3|    0|
|  4|    4|
|  5|    5|
|  6|    5|
|  7|    6|
|  8|    9|
|  9|    9|
| 10|   12|
| 11|   12|
| 12|   12|
| 13|   13|
| 14|   13|
| 15|   14|
| 16|   15|
| 17|   15|
| 18|   15|
| 19|   15|
+---+-----+
only showing top 20 rows



In [9]:
df_new.describe().toPandas()

,summary,day,cases
0,count,62,62
1,mean,30.5,140.24193548387098
2,stddev,18.041618552668716,297.9053477958609
3,min,0,0
4,max,61,1314


In [10]:
assembler = VectorAssembler(inputCols=['day'], outputCol='features')
df_ml = assembler.transform(df_new)
df_ml = df_ml.select(['features','cases'])
df_ml.show()

+--------+-----+
|features|cases|
+--------+-----+
|   [0.0]|    0|
|   [1.0]|    0|
|   [2.0]|    0|
|   [3.0]|    0|
|   [4.0]|    4|
|   [5.0]|    5|
|   [6.0]|    5|
|   [7.0]|    6|
|   [8.0]|    9|
|   [9.0]|    9|
|  [10.0]|   12|
|  [11.0]|   12|
|  [12.0]|   12|
|  [13.0]|   13|
|  [14.0]|   13|
|  [15.0]|   14|
|  [16.0]|   15|
|  [17.0]|   15|
|  [18.0]|   15|
|  [19.0]|   15|
+--------+-----+
only showing top 20 rows



In [11]:
dataset_splits = df_ml.randomSplit([0.7, 0.3])
train_df = dataset_splits[0]
test_df = dataset_splits[1]

In [12]:
lr = LinearRegression(featuresCol = 'features', labelCol='cases', 
                        maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)

In [13]:
print("Intercept: {}".format(lr_model.intercept))
print("Coefficient: {}".format(lr_model.coefficients.values))

Intercept: -131.58888048745675
Coefficient: [8.69345409]


In [14]:
modelsummary = lr_model.summary
print("Number of Iterations: {}".format(modelsummary.totalIterations))
print("Objective History: {}\n".format(modelsummary.objectiveHistory))
print("Explained Variance: {}".format(modelsummary.explainedVariance))
print("R Squared: {}".format(modelsummary.r2))
print("Root Mean Squared Error(RMSE): {}".format(modelsummary.rootMeanSquaredError))
print("Mean Absolute Error: {}".format(modelsummary.meanAbsoluteError))
print("Mean Squared Error: {}\n".format(modelsummary.meanSquaredError))
train_df.describe().show()
modelsummary.residuals.show()

Number of Iterations: 3
Objective History: [0.49999999999999994, 0.43897140616169017, 0.2987219587091187]

Explained Variance: 24359.841304896127
R Squared: 0.40389195955131885
Root Mean Squared Error(RMSE): 189.95007606982853
Mean Absolute Error: 124.93489920288282
Mean Squared Error: 36081.03139893364

+-------+------------------+
|summary|             cases|
+-------+------------------+
|  count|                46|
|   mean|111.26086956521739|
| stddev|248.74229723708552|
|    min|                 0|
|    max|              1314|
+-------+------------------+

+------------------+
|         residuals|
+------------------+
|131.58888048745675|
|122.89542639996802|
|105.50851822499058|
|100.81506413750185|
| 93.12161005001312|
|  84.4281559625244|
| 76.73470187503568|
| 71.04124778754695|
| 62.34779370005822|
|  56.6543396125695|
| 47.96088552508078|
| 39.26743143759205|
| 31.57397735010332|
|22.880523262614602|
|15.187069175125885|
| 7.493615087637153|
|-1.199838999851579|
| -9.8932930

In [15]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","cases","features").show(5)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="cases", metricName="r2")
print("R Squared(R2) on test dataset = {}".format(lr_evaluator.evaluate(lr_predictions)))

+------------------+-----+--------+
|        prediction|cases|features|
+------------------+-----+--------+
|-114.2019723124793|    0|   [2.0]|
|33.586747174829014|   15|  [19.0]|
|42.280201262317746|   15|  [20.0]|
| 94.44092578725011|   15|  [26.0]|
|137.90819622469374|   22|  [31.0]|
+------------------+-----+--------+
only showing top 5 rows

R Squared(R2) on test dataset = 0.3514909551772748


In [16]:
modelsummary.predictions.show()

+--------+-----+-------------------+
|features|cases|         prediction|
+--------+-----+-------------------+
|   [0.0]|  0.0|-131.58888048745675|
|   [1.0]|  0.0|-122.89542639996802|
|   [3.0]|  0.0|-105.50851822499058|
|   [4.0]|  4.0| -96.81506413750185|
|   [5.0]|  5.0| -88.12161005001312|
|   [6.0]|  5.0|  -79.4281559625244|
|   [7.0]|  6.0| -70.73470187503568|
|   [8.0]|  9.0| -62.04124778754695|
|   [9.0]|  9.0| -53.34779370005822|
|  [10.0]| 12.0|  -44.6543396125695|
|  [11.0]| 12.0| -35.96088552508078|
|  [12.0]| 12.0| -27.26743143759205|
|  [13.0]| 13.0| -18.57397735010332|
|  [14.0]| 13.0| -9.880523262614602|
|  [15.0]| 14.0|-1.1870691751258846|
|  [16.0]| 15.0|  7.506384912362847|
|  [17.0]| 15.0|  16.19983899985158|
|  [18.0]| 15.0|  24.89329308734031|
|  [21.0]| 15.0|  50.97365534980648|
|  [22.0]| 15.0|  59.66710943729518|
+--------+-----+-------------------+
only showing top 20 rows

